In [12]:
from konlpy.tag import Twitter
from glob import glob
from collections import Counter
from collections import defaultdict
from random import shuffle
from pprint import pprint
from scipy.sparse import dok_matrix
import numpy as np
import ujson as json
import pickle

files = glob("crawledResult/*")
twitter = Twitter()

In [13]:
with open(files[0]) as f:
    movies = json.load(f)

In [14]:
movies[0].keys()

dict_keys(['ratings', 'code', 'title'])

In [15]:
print(movies[12]['ratings'][0]['reple'])
print(movies[12]['ratings'][0]['score'])

배우들의 캐스팅도 좋고 내용도 재밌는 드라마.
10


In [16]:
print(twitter.morphs(movies[12]['ratings'][0]['reple']))

['배우', '들', '의', '캐스팅', '도', '좋', '고', '내용', '도', '재밌는', '드라마', '.']


In [ ]:
max_length = 100
ratingList = defaultdict(list)
i = 0
for file in files:
    with open(file) as f:
        movies = json.load(f)
    for movie in movies:
        ratings = movie['ratings']
        for rating in ratings:
            i += 1

            reple = rating['reple']
            score = (rating['score'] - 1) / 9
            words = twitter.morphs(reple)
            if len(words) <= max_length and len(words) > 1:
                ratingList[score].append(words)
                
print(i)

In [11]:
minNum = min([len(l) for l in ratingList.values()])
for key in ratingList:
    print("{} {}".format(key, len(ratingList[key])))

1.0 938145
0.4444444444444444 63190
0.7777777777777778 179862
0.8888888888888888 211854
0.6666666666666666 114473
0.0 222550
0.5555555555555556 78465
0.2222222222222222 33409
0.3333333333333333 36127
0.1111111111111111 31485


In [18]:
voca = Counter()
dataset = []
for score, reples in ratingList.items():
    for reple in reples[:minNum]:
        voca.update(reple)
        dataset.append((score, reple))

shuffle(dataset)

max_seq = max([len(seq[1]) for seq in dataset])
print(max_seq)

num_data = len(dataset)
num_train = int(num_data * 0.8)
num_test = num_data - num_train
print("num_train: {}, num_test: {}".format(num_train, num_test))
voca_num = 20000

93
num_train: 251880, num_test: 62970


In [19]:
index2voca = [v[0] for v in voca.most_common(voca_num - 1)]
index2voca.append('<UNK>')
voca2index = {v: i for i, v in enumerate(index2voca)}

dataset_X = dok_matrix((num_data, voca_num), dtype=np.int32)
dataset_y = np.zeros(num_data, dtype=np.float32)

for i, data in enumerate(dataset):
    dataset_y[i] = data[0]
    for j, word in enumerate(data[1]):
        dataset_X[i, voca2index.get(word, voca2index['<UNK>'])] += 1

dataset_X = dataset_X.tocsr()

train_X = dataset_X[:num_train]
train_y = dataset_y[:num_train]
test_X = dataset_X[num_train:]
test_y = dataset_y[num_train:]

with open("data_ml.pkl", "wb") as f:
    pickle.dump(index2voca, f)
    pickle.dump(voca2index, f)
    pickle.dump(train_X, f)
    pickle.dump(train_y, f)
    pickle.dump(test_X, f)
    pickle.dump(test_y, f)

In [20]:
index2voca = [v[0] for v in voca.most_common(voca_num - 2)]
index2voca.append('<UNK>')
index2voca.insert(0, '<PAD>')
voca2index = {v: i for i, v in enumerate(index2voca)}

max_len = 100
dataset_X = np.zeros((num_data, max_len), dtype=np.int32)
dataset_y = np.zeros(num_data, dtype=np.float32)

for i, data in enumerate(dataset):
    dataset_y[i] = data[0]
    for j, word in enumerate(data[1]):
        dataset_X[i, j] = voca2index.get(word, voca2index['<UNK>'])

train_X = dataset_X[:num_train]
train_y = dataset_y[:num_train]
test_X = dataset_X[num_train:]
test_y = dataset_y[num_train:]

with open("data_deep.pkl", "wb") as f:
    pickle.dump(index2voca, f)
    pickle.dump(voca2index, f)
    pickle.dump(train_X, f)
    pickle.dump(train_y, f)
    pickle.dump(test_X, f)
    pickle.dump(test_y, f)